In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
import torch
import optuna
from dnn_training.trainer import *
from dnn_training.hyperparameter_scan import *

ModuleNotFoundError: No module named 'dnn_training'

In [7]:
args = argparse.Namespace()
args.createStudy = False
args.input = "/grid_mnt/data_cms_upgrade/cuisset/supercls/alessandro_electrons/supercls-v15-sampleDump/superclsDumper_\*.root"
args.output = '/grid_mnt/data_cms_upgrade/cuisset/supercls/dnn_training/hyperparams-v4b'
args.device = "cuda:1"
args.studyName = "hp-v4"
args.nEpochs = 6
args.timeout = None
args.cmssw_release_path="/grid_mnt/vol_home/llr/cms/cuisset/hgcal/supercls/repoForJobs/CMSSW_14_1_X_2024-04-29-1100"
args.cmssw_py_config_path="/grid_mnt/vol_home/llr/cms/cuisset/hgcal/supercls/repoForJobs/runs/NewElectronsD98/rereco.py"
args.cmssw_py_config_path_harvesting="/grid_mnt/vol_home/llr/cms/cuisset/hgcal/supercls/repoForJobs/runs/NewElectronsD98/step4.py"
args.input_files=[f"/grid_mnt/data_cms_upgrade/cuisset/supercls/NewElectronsD98/v1-65772-mustache/step3_{i}.root" for i in range(1, 11)]
args.input_DQM_forComparison="/grid_mnt/data_cms_upgrade/cuisset/supercls/NewElectronsD98/v1-65772-mustache/DQM/DQM_V0001_R000000001__Global__CMSSW_X_Y_Z__RECO.root"


os.makedirs(args.output, exist_ok=True)

study_kwargs = dict(
        study_name=args.studyName,
        #pruner=optuna.pruners.PatientPruner(optuna.pruners.HyperbandPruner(min_resource=10, max_resource=args.nEpochs), patience=10),
        storage=optuna.storages.JournalStorage(optuna.storages.JournalFileStorage(args.output + f"/journal_optuna_{args.studyName}.log"),)
)
if args.createStudy:
    study = optuna.create_study(**study_kwargs)
else:
    study = optuna.load_study(**study_kwargs)

if "BATCH_SYSTEM" in os.environ: # disable multithreading on HTCondor (until we can set request_cpus to something intermediate between 1 and 8 on LLR T3)
    torch.set_num_threads(1)
    torch.set_num_interop_threads(1)

study.optimize(functools.partial(objective_reportInference, args=args), timeout=args.timeout)

/tmp/ipykernel_310396/3191267670.py:21: ExperimentalWarning: JournalStorage is experimental (supported from v3.1.0). The interface can change in the future.
  storage=optuna.storages.JournalStorage(optuna.storages.JournalFileStorage(args.output + f"/journal_optuna_{args.studyName}.log"),)


Starting trial nb 3


  0%|          | 0/6 [00:00<?, ?it/s]

[W 2024-05-03 19:24:43,344] Trial 3 failed with parameters: {'dropout': 0.18643026378856922, 'trainingLossType': 'continuousAssociationScore', 'lr': 0.004839046787519854, 'batchSize': 10697, 'weightSamples': False} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/grid_mnt/data_cms_upgrade/cuisset/conda/envs/ticlRegression-gpu/lib/python3.11/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/grid_mnt/vol_home/llr/cms/cuisset/hgcal/supercls/repoForJobs/dnn_training/../dnn_training/hyperparameter_scan.py", line 39, in objective_reportInference
    trainer.full_train(datasets["trainDataset"], datasets["valDataset"], nepochs=args.nEpochs)
  File "/grid_mnt/vol_home/llr/cms/cuisset/hgcal/supercls/repoForJobs/dnn_training/../dnn_training/trainer.py", line 590, in full_train
    self.train_loop(train_dataloader)
  File "/grid_mnt/vol_home/llr/cms/cuisset/h

KeyboardInterrupt: 

In [4]:
study = optuna.create_study(
    pruner=optuna.pruners.HyperbandPruner(),
    storage=optuna.storages.JournalStorage(optuna.storages.JournalFileStorage("/grid_mnt/data_cms_upgrade/cuisset/supercls/dnn_training/hyperparams-v1/journal_test_1.log"),))

/tmp/ipykernel_145720/2220379784.py:3: ExperimentalWarning: JournalStorage is experimental (supported from v3.1.0). The interface can change in the future.
  storage=optuna.storages.JournalStorage(optuna.storages.JournalFileStorage("/grid_mnt/data_cms_upgrade/cuisset/supercls/dnn_training/hyperparams-v1/journal_test_1.log"),))
[I 2024-04-08 14:38:26,047] A new study created in Journal with name: no-name-00984b9b-42b7-48af-b0f4-20d08a63845f


In [18]:
study.optimize(objective, n_trials=2)

Loading dataset...
Dataset loaded into RAM


  0%|          | 0/10 [00:00<?, ?it/s]

train_loss = 0.10285763357193434
val_loss = 0.10916735231876373
train_loss = 0.08264435252653457
val_loss = 0.09647541493177414
train_loss = 0.07759700448145886
val_loss = 0.1035563051700592
train_loss = 0.07444620232920182
val_loss = 0.09241978079080582
train_loss = 0.0724340887180131
val_loss = 0.07782204449176788
train_loss = 0.07051101348056384
val_loss = 0.10951804369688034
train_loss = 0.06946153203110064
val_loss = 0.09353321045637131
train_loss = 0.06841316473489185
val_loss = 0.10037089139223099
train_loss = 0.06729305179849365
val_loss = 0.09302803874015808
train_loss = 0.06625829325019834
val_loss = 0.09621752798557281


[I 2024-04-08 15:37:01,814] Trial 15 finished with value: 0.29885056614875793 and parameters: {'dropout': 0.16291726718736904, 'trainingLossType': 'binary', 'lr': 0.003320793707269728, 'batchSize': 965, 'weightSamples': True}. Best is trial 15 with value: 0.29885056614875793.


Loading dataset...
Dataset loaded into RAM


  0%|          | 0/10 [00:00<?, ?it/s]

train_loss = 0.16573507529474119
val_loss = 0.13470520079135895
train_loss = 0.09702740688951728
val_loss = 0.11574326455593109
train_loss = 0.08570478300948199
val_loss = 0.11789839714765549
train_loss = 0.08116929784091681
val_loss = 0.11118064820766449
train_loss = 0.07787169014826992
val_loss = 0.0954996794462204
train_loss = 0.07568169994707816
val_loss = 0.09493781626224518
train_loss = 0.07438785041003662
val_loss = 0.08022262901067734
train_loss = 0.07237083764300378
val_loss = 0.07997513562440872
train_loss = 0.07129589985642884
val_loss = 0.08059107512235641
train_loss = 0.06983752779063912
val_loss = 0.08799845725297928


[I 2024-04-08 15:42:20,408] Trial 16 finished with value: 0.2991444170475006 and parameters: {'dropout': 0.14819865692923473, 'trainingLossType': 'binary', 'lr': 0.0010095301254578934, 'batchSize': 4922, 'weightSamples': True}. Best is trial 15 with value: 0.29885056614875793.
